In [1]:
####### PART 0: Load Packages
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
import time
import datetime as dt

data_folder_path = r'C:\Users\xmeng5\OneDrive - MetLife\Desktop\Pet\Hack_Data\\'
data_folder_path2 = r'C:\Users\xmeng5\OneDrive - MetLife\Desktop\Pet\data\\'

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
pd.options.display.max_seq_items = 2000
pd.set_option('display.float_format', lambda x: '%.3f' % x)



# In[3]:

#internal lead data load and cleanse
leads = pd.read_csv(data_folder_path2 + 'PetFirst_Leads_Masked.csv')
leads = leads.loc[leads['createdate'].str.len()!=3]
leads['createdate'] = pd.to_datetime(leads['createdate'],infer_datetime_format=True)
leads['soldpolicy'].replace({'False':0, False:0, True:1, 'True':1},inplace=True)
#leads['month'] = leads['createdate'].dt.to_period('M')
leads['year'] = pd.DatetimeIndex(leads['createdate']).year

leads = leads[leads['createdate'] > '01-01-2019']
leads = leads[(leads['pet1age'].astype(str).str[0].str.isdigit())|leads['pet1age'].isnull()]
#leads.drop_duplicates(subset=['email','zipcode','year'],inplace=True)
#leads.shape, leads['soldpolicy'].mean()


# In[4]:


leads['petscovered'] = leads['petscovered'].astype(int)

#pet1/2/3age can be missing, try derive pet age with petbday. Create pet1/2/3age_combo using derived age to fill missing ages
leads[['pet1age','pet2age','pet3age']] = leads[['pet1age','pet2age','pet3age']].astype(float)
leads['pet1age_derive'] = pd.to_datetime(leads['createdate'],infer_datetime_format=True) - pd.to_datetime(leads['pet1bday'],infer_datetime_format=True)
leads['pet1age_derive'] = leads['pet1age_derive'].dt.days / 365
leads['pet2age_derive'] = pd.to_datetime(leads['createdate'],infer_datetime_format=True) - pd.to_datetime(leads['pet2bday'],infer_datetime_format=True)
leads['pet2age_derive'] = leads['pet2age_derive'].dt.days / 365
leads['pet3age_derive'] = pd.to_datetime(leads['createdate'],infer_datetime_format=True) - pd.to_datetime(leads['pet3bday'],infer_datetime_format=True)
leads['pet3age_derive'] = leads['pet3age_derive'].dt.days / 365
leads['pet1age_combo'] = np.floor(leads['pet1age'].fillna(leads['pet1age_derive']))
leads['pet2age_combo'] = np.floor(leads['pet2age'].fillna(leads['pet2age_derive']))
leads['pet3age_combo'] = np.floor(leads['pet3age'].fillna(leads['pet3age_derive']))


us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

leads['state'].replace(us_state_abbrev, inplace=True)
leads['has_dog'] = leads['pet1species'].isin(['Dog','1'])|leads['pet2species'].isin(['Dog','1'])|leads['pet3species'].isin(['Dog','1'])
leads['has_cat'] = leads['pet1species'].isin(['Cat','2'])|leads['pet2species'].isin(['Cat','2'])|leads['pet3species'].isin(['Cat','2'])


# In[10]:


leads['pet1gender'].replace({'1':'M', 'Male':'M','2':'F',2:'F','Female':'F'},inplace=True)
leads['pet2gender'].replace({'1':'M', 'Male':'M','2':'F',2:'F','Female':'F'},inplace=True)
leads['pet3gender'].replace({'1':'M', 'Male':'M','2':'F',2:'F','Female':'F'},inplace=True)


# In[12]:


ax_col = ['IN_RECORDNO',
          'D_RM_PRTY_HASH_KEY_NEW',
          'D_PE_9509_1ST_EDUC_CD',
          'D_PE_8610_1ST_GNDR_CD',
          'D_PE_8653_ONLINE_PUR_IND',
          'D_PE_7744_TRAVEL_US_IND',
          'D_PE_8609_HH_MRY_CD',
          'D_PE_8622_CHILD_IND',         
          'D_PE_7110_ECON_STAB_FIN_CD',
          'D_PE_8606_DWELL_OCCUPY_CD',
          'D_PE_7737_MAGAZINE_IND',
          'D_PE_8616_1ST_AGE_2YR_CD',
          'D_PE_1270_PERSONICA_CLUSTER_CD',
          'D_PE_1270_PERSONICA_PRECISION',
          'ACL_MATCH']


# In[13]:


#load Acxiom data, enable/disable usecols argument to limit number of columns
leads_ax = pd.read_csv(data_folder_path2 + 'ACXIOM_DDAM_ADHOC_TPAPETPROJECTLEA_20201116_0000001.TXT', sep='|')
#leads_ax = pd.read_csv(box_base + 'ACXIOM_DDAM_ADHOC_TPAPETPROJECTLEA_20201116_0000001.TXT', sep='|',usecols=ax_col)


# In[15]:
leads_merge = pd.merge(leads,leads_ax,how='left',left_on='leadid', right_on='IN_RECORDNO').drop_duplicates(subset='leadid')
leads_merge['id'] = 1

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (11,14,18,19,20,37,38,39,40,42,43,46,47,48,49,50,51,52,53,54,55,56,57,59,60,61,62,63,64,67,68) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (34,35,36,56,77,80,83,123,175,176,178,244,245,251,323,654,655,656,657,658,659,660) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
# del leads_merge
# leads_merge = pd.merge(leads,leads_ax,how='left',left_on='leadid', right_on='IN_RECORDNO').drop_duplicates(subset='leadid')
# leads_merge['id'] = 1

In [2]:
leads_merge_copy = leads_merge.copy()

In [ ]:
leads_merge.columns

In [3]:
#Mark Data Prep
def binVar_append(inpdat, binVar, binValueList, varname_prefix,varname_suffix):   
    for num in range(len(binValueList)):
        dummy = np.where(inpdat['%s' % binVar] == binValueList[num],1,0)
        inpdat['%s' % varname_prefix + varname_suffix[num]] = dummy

def Missing_append(inpdat, binVar):   
        dummy = np.where(inpdat['%s' % binVar].isnull()==1,1,0)
        inpdat['%s' % binVar + '_missing'] = dummy

In [4]:
#plan
binVar_append(inpdat = leads_merge, binVar = "plan",
                          binValueList = ["Accident & Illness $1000","Accident & Illness $2000",
                                          "Accident & Illness $3000","Accident & Illness $4000",
                                          "Accident & Illness $5000","Accident & Illness 30-Day",
                                          "Premier $1000","Premier $2000","Premier $5000","Standard 30-Day"],
                          varname_prefix = "Plan_", 
                          varname_suffix = ["AI_1000","AI_2000",
                                          "AI_3000","AI_4000",
                                          "AI_5000","AI_30D",
                                          "P_1000","P_2000","P_5000","S_30D"])
Missing_append(inpdat = leads_merge,binVar = "plan")

leads_merge = leads_merge.drop(['plan'], axis = 1)

#plancopay
check = pd.DataFrame(pd.crosstab(leads_merge['plancopay'],leads_merge['id']))
binVar_append(inpdat = leads_merge, binVar = "plancopay",
                          binValueList = ["100%","80%",
                                          "90%"],
                          varname_prefix = "PlanCoPay_", 
                          varname_suffix = ["100","80","90"])
Missing_append(inpdat = leads_merge,binVar = "plancopay")

leads_merge = leads_merge.drop(['plancopay'], axis = 1)

#Annual/Monthly
check = pd.DataFrame(pd.crosstab(leads_merge['annual/monthly'],leads_merge['id']))
binVar_append(inpdat = leads_merge, binVar = "annual/monthly",
                          binValueList = ["Annual","Monthly"],
                          varname_prefix = "Pay_", 
                          varname_suffix = ["Annual","Monthly"])
Missing_append(inpdat = leads_merge,binVar = "annual/monthly")

leads_merge = leads_merge.drop(['annual/monthly'], axis = 1)

#PaymentType
check = pd.DataFrame(pd.crosstab(leads_merge['paymenttype'],leads_merge['id']))
binVar_append(inpdat = leads_merge, binVar = "paymenttype",
                          binValueList = ["Check(ACH)","Credit Card","PetFirst","VisFin"],
                          varname_prefix = "Type_", 
                          varname_suffix = ["check","credit","petfirst","visfin"])
Missing_append(inpdat = leads_merge,binVar = "paymenttype")

leads_merge = leads_merge.drop(['paymenttype'], axis = 1)

#EmailDeals
check = pd.DataFrame(pd.crosstab(leads_merge['emaildeals'],leads_merge['soldpolicy']))
binVar_append(inpdat = leads_merge, binVar = "emaildeals",
                          binValueList = [False,True],
                          varname_prefix = "emaildeal_", 
                          varname_suffix = ["No","Yes"])
leads_merge['EmailDeal'] = np.where(leads_merge['emaildeal_No']==1,0,1)

check = pd.DataFrame(pd.crosstab(leads_merge['EmailDeal'],leads_merge['id']))
leads_merge = leads_merge.drop(['emaildeals','emaildeal_No','emaildeal_Yes'], axis = 1)

#PetsCovered
check = pd.DataFrame(pd.crosstab(leads_merge['petscovered'],leads_merge['id']))
binVar_append(inpdat = leads_merge, binVar = "petscovered",
                          binValueList = [0,1,2,3],
                          varname_prefix = "PetsCovered_", 
                          varname_suffix = ["None","One","Two","Three"])
Missing_append(inpdat = leads_merge,binVar = "petscovered")

leads_merge = leads_merge.drop(['petscovered'], axis = 1)

#PetBreed
check = pd.DataFrame(pd.crosstab(leads_merge['pet3breed'],leads_merge['id']))
leads_merge['breed1lower'] = leads_merge['pet1breed'].str.lower()
leads_merge['mix'] = np.where(leads_merge['breed1lower'].str.contains('mix', regex = False) == 1,1,0)
leads_merge['domestic'] = np.where(leads_merge['breed1lower'].str.contains('domestic', regex = False) == 1,1,0)
leads_merge['Mutt_Code'] = np.where(leads_merge['breed1lower'].isin(['2','54','569','575','576','581','585','586']),1,0)
leads_merge['Mutt_Breed'] = leads_merge[['mix', 'domestic','Mutt_Code']].max(axis=1)
Missing_append(inpdat = leads_merge,binVar = "breed1lower")
leads_merge['Pure_Breed'] = np.where((leads_merge['Mutt_Breed'] == 0) & (leads_merge['breed1lower_missing']==0),1,0)
leads_merge = leads_merge.drop(['breed1lower','mix','domestic','Mutt_Code','pet1breed','pet2breed','pet3breed'], axis = 1)

#pet1gender
check = pd.DataFrame(pd.crosstab(leads_merge['pet1gender'],leads_merge['id']))
binVar_append(inpdat = leads_merge, binVar = "pet1gender",
                          binValueList = ['F','M'],
                          varname_prefix = "FirstPetGender_", 
                          varname_suffix = ["F","M"])
Missing_append(inpdat = leads_merge,binVar = "pet1gender")

leads_merge = leads_merge.drop(['pet1gender','pet2gender','pet3gender'], axis = 1)

#ishubspotmigrated
check = pd.DataFrame(pd.crosstab(leads_merge['ishubspotmigrated'],leads_merge['id']))
binVar_append(inpdat = leads_merge, binVar = 'ishubspotmigrated',
                          binValueList = [False,True],
                          varname_prefix = "HubSpotMigrate_", 
                          varname_suffix = ["No","Yes"])
leads_merge['HubSpotMigrate'] = np.where(leads_merge['HubSpotMigrate_No']==1,0,1)
check = pd.DataFrame(pd.crosstab(leads_merge['HubSpotMigrate'],leads_merge['id']))

leads_merge = leads_merge.drop(['ishubspotmigrated','HubSpotMigrate_No','HubSpotMigrate_Yes'], axis = 1)
#iscontactinvalid
check = pd.DataFrame(pd.crosstab(leads_merge['iscontactinvalid'],leads_merge['id']))
binVar_append(inpdat = leads_merge, binVar = 'iscontactinvalid',
                          binValueList = [False,True],
                          varname_prefix = "InvalidContact_", 
                          varname_suffix = ["No","Yes"])
leads_merge['InvalidContact'] = np.where(leads_merge['InvalidContact_No']==1,0,1)
check = pd.DataFrame(pd.crosstab(leads_merge['InvalidContact'],leads_merge['id']))

leads_merge = leads_merge.drop(['iscontactinvalid','InvalidContact_No','InvalidContact_Yes'], axis = 1)

#Source
check = pd.DataFrame(pd.crosstab(leads_merge['source'],leads_merge['id']))
binVar_append(inpdat = leads_merge, binVar = 'source',
                          binValueList = ["Get Quote Bottom CTA","Consumer\'s Advocates",
                                          "Natural Intelligence Desktop","Emily Emails",
                                          "Brand X Ads","google Search Campaign",
                                          "Pet Insurance Review Lead Source",
                                          "Natural Intelligence Mobile", "Natural Intelligence_New Site"],
                          varname_prefix = "Source_", 
                          varname_suffix = ["GetQuote_CTA","ConsumerAdvocates",
                                          "NatIntelDesktop","EmilyEmails",
                                          "BrandXAds","googleSearchCampaign",
                                          "PetInsuranceReview",
                                          "NaturalIntelligenceMobile", "NaturalIntelligenceNewSite"])
Missing_append(inpdat = leads_merge,binVar = "source")


leads_merge = leads_merge.drop(['source'], axis = 1)

#Age
check = pd.DataFrame(pd.crosstab(leads_merge['pet1age'],leads_merge['id']))
leads_merge['Oldest_Pet'] = leads_merge[['pet1age', 'pet2age','pet3age']].max(axis=1)
leads_merge['Youngest_Pet'] = leads_merge[['pet1age', 'pet2age','pet3age']].min(axis=1)
leads_merge['Average_Pet_Age'] = leads_merge[['pet1age', 'pet2age','pet3age']].mean(axis=1)
leads_merge['Median_Pet_Age'] = leads_merge[['pet1age', 'pet2age','pet3age']].median(axis=1)

#emailquote
check = pd.DataFrame(pd.crosstab(leads_merge['emailquote'],leads_merge['id']))
binVar_append(inpdat = leads_merge, binVar = 'emailquote',
                          binValueList = [False,True],
                          varname_prefix = "EmailQuote_", 
                          varname_suffix = ["No","Yes"])
leads_merge['EmailQuote'] = np.where(leads_merge['EmailQuote_No']==1,0,1)
check = pd.DataFrame(pd.crosstab(leads_merge['EmailQuote'],leads_merge['id']))

leads_merge = leads_merge.drop(['emailquote','EmailQuote_No','EmailQuote_Yes'], axis = 1)

#has_cat
leads_merge['has_cat'] = np.where(leads_merge['has_cat']==False,0,1)

#has_dog
leads_merge['has_dog'] = np.where(leads_merge['has_dog']==False,0,1)

#Acxiom Fields

#EDUCATION
check = pd.DataFrame(pd.crosstab(leads_merge['D_PE_9509_1ST_EDUC_CD'],leads_merge['id']))
binVar_append(inpdat = leads_merge, binVar = "D_PE_9509_1ST_EDUC_CD",
                          binValueList = [1,2,3,4],
                          varname_prefix = "EDUC_", 
                          varname_suffix = ["HS","COL","GRAD","VOC"])
Missing_append(inpdat = leads_merge,binVar = "D_PE_9509_1ST_EDUC_CD")

leads_merge = leads_merge.drop(['D_PE_9509_1ST_EDUC_CD'], axis = 1)

#PRIMARY PERSON GENDER
check = pd.DataFrame(pd.crosstab(leads_merge['D_PE_8610_1ST_GNDR_CD'],leads_merge['id']))
binVar_append(inpdat = leads_merge, binVar = "D_PE_8610_1ST_GNDR_CD",
                          binValueList = ['F','M','U'],
                          varname_prefix = "PRIM_GEN_", 
                          varname_suffix = ["F","M","U"])
Missing_append(inpdat = leads_merge,binVar = "D_PE_8610_1ST_GNDR_CD")

leads_merge = leads_merge.drop(['D_PE_8610_1ST_GNDR_CD'], axis = 1)

#ONLINE PURCHASE
Missing_append(inpdat = leads_merge,binVar = "D_PE_8653_ONLINE_PUR_IND")

leads_merge = leads_merge.drop(['D_PE_8653_ONLINE_PUR_IND'], axis = 1)

#D_PE_7744_TRAVEL_US_IND (Leave it alone)

#MARITAL STATUS
check = pd.DataFrame(pd.crosstab(leads_merge['D_PE_8609_HH_MRY_CD'],leads_merge['id']))
binVar_append(inpdat = leads_merge, binVar = "D_PE_8609_HH_MRY_CD",
                          binValueList = ['A','B','M','S'],
                          varname_prefix = "M_STATUS_", 
                          varname_suffix = ["INF_MAR","INF_SING","MAR","SING"])
Missing_append(inpdat = leads_merge,binVar = "D_PE_8609_HH_MRY_CD")

leads_merge = leads_merge.drop(['D_PE_8609_HH_MRY_CD'], axis = 1)

#D_PE_8622_CHILD_IND
check = pd.DataFrame(pd.crosstab(leads_merge['D_PE_8622_CHILD_IND'],leads_merge['id']))
binVar_append(inpdat = leads_merge, binVar = "D_PE_8622_CHILD_IND",
                          binValueList = ['Y','N'],
                          varname_prefix = "KIDS_", 
                          varname_suffix = ["Yes","No"])
Missing_append(inpdat = leads_merge,binVar = "D_PE_8622_CHILD_IND")

leads_merge = leads_merge.drop(['D_PE_8622_CHILD_IND'], axis = 1)

#D_PE_7110_ECON_STAB_FIN_CD (Leave it alone)

#HOMEOWNERSHIP
check = pd.DataFrame(pd.crosstab(leads_merge['D_PE_8606_DWELL_OCCUPY_CD'],leads_merge['id']))
binVar_append(inpdat = leads_merge, binVar = "D_PE_8606_DWELL_OCCUPY_CD",
                          binValueList = ['O','R'],
                          varname_prefix = "HOME_", 
                          varname_suffix = ["OWNER","RENTER"])
Missing_append(inpdat = leads_merge,binVar = "D_PE_8606_DWELL_OCCUPY_CD")

leads_merge = leads_merge.drop(['D_PE_8606_DWELL_OCCUPY_CD'], axis = 1)

#D_PE_7737_MAGAZINE_IND
Missing_append(inpdat = leads_merge,binVar = "D_PE_7737_MAGAZINE_IND")
leads_merge = leads_merge.drop(['D_PE_7737_MAGAZINE_IND'], axis = 1)

#D_PE_8616_1ST_AGE_2YR_CD 
Missing_append(inpdat = leads_merge,binVar = "D_PE_8616_1ST_AGE_2YR_CD")

#D_PE_1270_PERSONICA_CLUSTER_CD
check = pd.DataFrame(pd.crosstab(leads_merge['D_PE_1270_PERSONICA_CLUSTER_CD'],leads_merge['id']))
binVar_append(inpdat = leads_merge, binVar = "D_PE_1270_PERSONICA_CLUSTER_CD",
                          binValueList = [13,5,14,26,7,6,10],
                          varname_prefix = "AX_CLUSTER_", 
                          varname_suffix = ["WORKPLAY","ACTIVEINVOLVED","CAREERCENTERED",
                                            "GETTINGESTABLISHED","ACTIVELIFE","CASUALCOMFORT","CAREERTRAVEL"])
Missing_append(inpdat = leads_merge,binVar = "D_PE_1270_PERSONICA_CLUSTER_CD")

leads_merge = leads_merge.drop(['D_PE_1270_PERSONICA_CLUSTER_CD'], axis = 1)

#TIME VARIABLES

leads_merge['DATE'] = pd.to_datetime(leads_merge['createdate'],format = "%Y-%m-%d").dt.date

leads_merge['month'] = pd.DatetimeIndex(leads_merge['DATE']).month
leads_merge['day_of_week'] = pd.DatetimeIndex(leads_merge['DATE']).weekday
leads_merge['quarter'] = pd.DatetimeIndex(leads_merge['DATE']).quarter

check = pd.DataFrame(pd.crosstab(leads_merge['month'],leads_merge['id']))
binVar_append(inpdat = leads_merge, binVar = "month",
                          #binValueList = ['1','2','3','4','5','6','7','8','9','10','11','12'],
                          binValueList = [1,2,3,4,5,6,7,8,9,10,11,12],
                          varname_prefix = "Month_", 
                          varname_suffix = ["JAN","FEB","MAR",
                                            "APR","MAY","JUN","JUL",
                                            "AUG","SEPT","OCT","NOV","DEC"])
binVar_append(inpdat = leads_merge, binVar = "day_of_week",
                          #binValueList = ['0','1','2','3','4','5','6'],
                          binValueList = [0,1,2,3,4,5,6],
                          varname_prefix = "Day_", 
                          varname_suffix = ["MON","TUES","WED",
                                            "THURS","FRI","SAT","SUN"])
binVar_append(inpdat = leads_merge, binVar = "quarter",
                          #binValueList = ['1','2','3','4'],
                          binValueList = [1,2,3,4],
                          varname_prefix = "Quarter_", 
                          varname_suffix = ["ONE","TWO","THREE",
                                            "FOUR"])

leads_merge = leads_merge.drop(['month','day_of_week','quarter','DATE','createdate'], axis = 1)

#GEOGRAPHY
check = pd.DataFrame(pd.crosstab(leads_merge['state'],leads_merge['id']))
leads_merge['region'] = np.where(leads_merge['state'].isin(['CA','OR','WA','HI']),'REGION_WEST',
                        np.where(leads_merge['state'].isin(['AZ','NM','NV']),'REGION_DESERT',
                            np.where(leads_merge['state'].isin(['ID','MT','UT','WY','CO','AK']), 'REGION_ROCKIES', 
                                     np.where(leads_merge['state'].isin(['ND','SD','KS','NE','MO']), 'REGION_PLAINS', 
                                         np.where(leads_merge['state'].isin(['OK','TX']), 'REGION_TX',  
                                                  np.where(leads_merge['state'].isin(['AR','LA','MS','AL','GA','TN','KY','SC','VA','WV','NC']), 'REGION_SOUTHEAST',
                                                         np.where(leads_merge['state'].isin(['FL']), 'REGION_FL', 
                                                                 np.where(leads_merge['state'].isin(['MN','IA','WI','MI']), 'REGION_UPPER_MW', 
                                                                     np.where(leads_merge['state'].isin(['IL','IN','OH']), 'REGION_HEARTLAND', 
                                                                         np.where(leads_merge['state'].isin(['DC','MD','DE','PA','NJ']),'REGION_MID_ATL', 
                                                                            np.where(leads_merge['state'].isin(['NY']),'REGION_NY', 
                                                                               np.where(leads_merge['state'].isin(['VT','NH','MA','RI','CT','ME']),'REGION_NEW_ENGLAND','REGION_OTHER'))))))))))))
check = pd.DataFrame(pd.crosstab(leads_merge['soldpolicy'],leads_merge['id']))
binVar_append(inpdat = leads_merge, binVar = "region",
                          binValueList = ['REGION_WEST','REGION_DESERT','REGION_ROCKIES','REGION_PLAINS','REGION_TX','REGION_SOUTHEAST','REGION_FL','REGION_UPPER_MW',
                                          'REGION_HEARTLAND','REGION_MID_ATL','REGION_NY','REGION_NEW_ENGLAND','REGION_OTHER'],
                          varname_prefix = "Region_", 
                          varname_suffix = ["West","Desert","Rockies",
                                            "Plains","Texas","SouthEast","Florida",
                                            "UpperMW","Heartland","MID_ATL","NY","NE","OTHER"])
leads_merge = leads_merge.drop(['state','region'], axis = 1)

#soldpolicy
leads_merge['soldpolicy'] = np.where(leads_merge['soldpolicy'].isnull()==1,0,leads_merge['soldpolicy'])
leads_merge['soldpolicy'] = np.where(leads_merge['soldpolicy']==1,1,0)


In [ ]:
#del leads_merge_copy
del leads_ax

In [ ]:
LM2 = leads_merge.drop(['company','firstname','lastname','phonenumber','email',
                        'streetaddress','city','zipcode','quoteid',
                        'enrollmentcode','heardoption','completed',
                        'pet1name','pet1bday','pet1species','pet2name',
                        'pet2bday','pet2species','pet3name','pet3bday',
                        'pet3species','email1date','email2date',
                        'email3date','unsubscribe','openedemail','numberofclicks',
                        'coverage','loyaltycard','workphone','cellphone',
                        'streetaddress2','additionalinfo','etsubscriberid',
                        'iscontactduplicate','isreported','pet1age','pet2age',
                        'pet3age','pet1color','pet2color','pet3color',
                        'laststep','pet1size','pet2size','pet3size',
                        'discount','issms','leadguidid','year','pet1age_derive',
                        'pet2age_derive','pet3age_derive','pet1age_combo',
                        'pet2age_combo','pet3age_combo','IN_RECORDNO',
                        'D_RM_PRTY_HASH_KEY_NEW','ACL_MATCH','id','D_PE_1270_PERSONICA_PRECISION'],axis = 1)

#SPLIT FOR MODELING
LM_train, LM_test = train_test_split(LM2, test_size = 0.2, random_state=42)

ID_train = LM_train[['leadid']]
ID_test = LM_test[['leadid']]

X_train = LM_train.drop(['leadid','soldpolicy'], axis = 1)
Y_train = LM_train[['soldpolicy']]
X_train = X_train.replace(float("NaN"),0)
X_test = LM_test.drop(['leadid','soldpolicy'], axis = 1)
Y_test = LM_test[['soldpolicy']]
X_test = X_test.replace(float("NaN"),0)

#MODEL The Data

#Gradient Boosting Classifier
GBC_obj= GradientBoostingClassifier(max_depth = 6, n_estimators = 150, learning_rate = 0.1)
GBC_obj.fit(X_train,Y_train)
GBC_val_scores = GBC_obj.predict_proba(X_test)
GBC_val_scores2 = GBC_val_scores[:,1]
# GBC_roc = roc_auc_score(Y_test, GBC_val_scores2)
# print(GBC_roc)

# check = pd.DataFrame(GBC_obj.feature_importances_)
# check.columns = ['Feature_Importance']
# pred_cols = pd.DataFrame(X_train.columns)
# pred_cols.columns = ['Predictor']
# varimp = pd.concat([pred_cols,check], axis = 1)
# varimp['FI_Rank']  = varimp['Feature_Importance'].rank(ascending = False)

# #Random Forest
# from sklearn.ensemble import RandomForestClassifier
# forest_obj = RandomForestClassifier(n_estimators = 500, max_leaf_nodes = 16, n_jobs = -1)
# forest_obj.fit(X_train,Y_train)
# forest_val_scores = forest_obj.predict_proba(X_test)
# forest_val_scores2 = forest_val_scores[:,1]
# forest_roc = roc_auc_score(Y_test, forest_val_scores2)
# print(forest_roc)

# check = pd.DataFrame(forest_obj.feature_importances_)
# check.columns = ['Feature_Importance']
# pred_cols = pd.DataFrame(X_train.columns)
# pred_cols.columns = ['Predictor']
# varimp = pd.concat([pred_cols,check], axis = 1)
# varimp['FI_Rank']  = varimp['Feature_Importance'].rank(ascending = False)

# #Logistic Regression
# from sklearn.linear_model import LogisticRegression

# lr_obj = LogisticRegression()
# lr_obj.fit(X_train,Y_train)
# lr_val_scores = lr_obj.predict_proba(X_test)
# lr_val_scores2 = lr_val_scores[:,1]
# lr_roc = roc_auc_score(Y_test, lr_val_scores2)
# print(lr_roc)
